In [1]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:

In [2]:
!pip install -U sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [3]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.2 MB/s eta 0:00:00


In [4]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"

In [5]:
os.environ["GROQ_API_KEY"]="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"

In [6]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [8]:
from sentence_transformers import SentenceTransformer
import torch

class CustomEmbeddings:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = SentenceTransformer(model_name, device=device)
        self.device = device

    def embed_documents(self, documents):
        if isinstance(documents[0], str):
            texts = documents
        else:
            texts = [doc.page_content for doc in documents]

        embeddings = self.model.encode(texts, convert_to_tensor=True, device=self.device)
        return embeddings.cpu().numpy().tolist()  # Ensure embeddings are returned as a list

    def embed_query(self, query):
        embedding = self.model.encode([query], convert_to_tensor=True, device=self.device)
        return embedding.cpu().numpy().tolist()[0]  # Ensure the embedding is a list

# Initialize your custom embedding class
custom_embeddings = CustomEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Test data
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

# Embed the question and document
question_embedding = custom_embeddings.embed_query(question)
document_embedding = custom_embeddings.embed_query(document)
print(len(question_embedding))
print(len(document_embedding))

384
384


In [10]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(question_embedding, document_embedding)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.57826574726501


In [11]:
!pip install PyMuPDF
!pip install faiss-cpu
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 46.3 MB/s eta 0:00:00


In [12]:
import fitz  # PyMuPDF

def extract_text_from_pdfs(pdf_paths):
    texts = []
    for pdf_path in pdf_paths:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            texts.append(page.get_text())
    return texts

pdf_paths = ['debra_satz_why_some_things_should_not_be_for_sale.pdf', 'The_Luxury_Strategy_Break_the_Rules_of_Marketing_to_Build_Luxury_Brands.pdf']  # Replace with your PDF paths
pdf_texts = extract_text_from_pdfs(pdf_paths)

In [13]:
len(pdf_texts)

602

In [14]:
from langchain.schema import Document

pdf_documents = [Document(page_content=text) for text in pdf_texts]

In [16]:
len(pdf_documents)

602

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

pdf_splits = text_splitter.split_documents(pdf_documents)

print(len(pdf_splits))

1643


Part 3: Retrieval

In [21]:
# Index
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=pdf_splits,
                                    embedding=custom_embeddings)

retriever = vectorstore.as_retriever()

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [23]:
docs = retriever.get_relevant_documents("Can you tell me about the non-return effect?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [24]:
len(docs)

1

In [25]:
docs[0]

Document(page_content='The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nincrease in spending power, globalization and communications – when they\nare all working at full throttle they help propel luxury to previously\nunknown levels.\nBefore moving on to other things, we need to add a word about a well-\nknown effect of luxury, namely the ‘non-return effect’, or ‘ratchet effect’, the\ncounterpart of the ‘non-linear increase’ that we talked about earlier: once\npeople have tasted luxury in whatever area, it is very difficult for them to turn\naway from it – to come back to earth. When there is a fall in spending power,\nwe witness a symmetrical phenomenon to the one that we identified when\nspending power was increased: people start cutting their expenditure on\nevery kind of conventional product (for example, they will trade their\nChevrolet for a smaller and more economic Ford), but it’s a different matter\nwhen it comes to luxury pro

In [26]:
docs[0].page_content

'The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nincrease in spending power, globalization and communications – when they\nare all working at full throttle they help propel luxury to previously\nunknown levels.\nBefore moving on to other things, we need to add a word about a well-\nknown effect of luxury, namely the ‘non-return effect’, or ‘ratchet effect’, the\ncounterpart of the ‘non-linear increase’ that we talked about earlier: once\npeople have tasted luxury in whatever area, it is very difficult for them to turn\naway from it – to come back to earth. When there is a fall in spending power,\nwe witness a symmetrical phenomenon to the one that we identified when\nspending power was increased: people start cutting their expenditure on\nevery kind of conventional product (for example, they will trade their\nChevrolet for a smaller and more economic Ford), but it’s a different matter\nwhen it comes to luxury products (they will keep 

In [27]:
len(docs[0].page_content)

1248

In [29]:
docs[0].page_content[:100]

'The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nin'

Part 4: Generation

In [30]:
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'))])

In [31]:
# LLM
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# Chain
chain = prompt | llm

# Run
chain.invoke({"context":docs,"question":"Can you tell me about the non-return effect?"})

AIMessage(content="Yes, based on the document provided, the 'non-return effect' or 'ratchet effect' refers to the phenomenon where, once people have experienced luxury in a certain area, it becomes very difficult for them to give it up. Even when there is a decrease in their spending power, they tend to cut back on conventional products but are reluctant to let go of their luxury items. They may even forego other necessities as long as they can afford to maintain their luxury possessions, such as a mansion or a high-end car. This effect is symmetrical to the 'non-linear increase' observed when spending power is increased.", response_metadata={'token_usage': {'completion_tokens': 133, 'prompt_tokens': 351, 'total_tokens': 484, 'completion_time': 0.214004185, 'prompt_time': 0.020074392, 'queue_time': None, 'total_time': 0.234078577}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-0566e97a-3732-4b1c-8fb4-b494c